# Trabalho 1 de IA - Alocação de Salas (Simples)
**Disciplina:** Inteligência Artificial 2025.2
**Professor:** Lucídio Cabral
**Tema:** 6. Alocação de salas (Timetabling Simplificado)

---
### 1. Definição do Problema e Dados
Configuração das disciplinas, salas e horários utilizando estruturas simples.

In [1]:
import heapq
import time
from copy import copy

# --- 1. CONFIGURAÇÃO DO AMBIENTE (VERSÃO "PONTO IDEAL") ---

# 4 Horários (Aumenta o fator de ramificação)
HORARIOS = ["08:00", "10:00", "14:00", "16:00"]

# 6 Salas: Poucas salas grandes (o recurso escasso)
SALAS = [
    {'id': 'S_Gigante1', 'cap': 60}, # Ouro: Todo mundo quer, mas só cabe as grandes aqui
    {'id': 'S_Gigante2', 'cap': 60}, 
    {'id': 'S_Media1',   'cap': 40},
    {'id': 'S_Media2',   'cap': 40},
    {'id': 'Lab_Peq1',   'cap': 20},
    {'id': 'Lab_Peq2',   'cap': 20}
]

# 10 Disciplinas ordenadas para "enganar" o algoritmo (Pequenas primeiro, Grandes depois)
DISCIPLINAS = [
    # --- As Fáceis (O algoritmo vai gastar as salas grandes aqui se não for esperto) ---
    {'id': 'D01', 'nome': 'Optativa 1',   'prof': 'Prof_A', 'alunos': 15},
    {'id': 'D02', 'nome': 'Optativa 2',   'prof': 'Prof_B', 'alunos': 15},
    {'id': 'D03', 'nome': 'Optativa 3',   'prof': 'Prof_C', 'alunos': 18},
    {'id': 'D04', 'nome': 'Seminários',   'prof': 'Prof_D', 'alunos': 20},
    
    # --- As Médias (Começam a apertar) ---
    {'id': 'D05', 'nome': 'Redes',        'prof': 'Prof_A', 'alunos': 35}, # Conflito Prof_A
    {'id': 'D06', 'nome': 'Banco Dados',  'prof': 'Prof_B', 'alunos': 35}, # Conflito Prof_B
    {'id': 'D07', 'nome': 'Eng. Soft',    'prof': 'Prof_E', 'alunos': 40},

    # --- As Pesadas (Só cabem nas S_Gigante, mas elas podem estar ocupadas pelas pequenas) ---
    {'id': 'D08', 'nome': 'I.A.',         'prof': 'Lucidio','alunos': 55}, 
    {'id': 'D09', 'nome': 'Cálculo 1',    'prof': 'Prof_F', 'alunos': 58},
    {'id': 'D10', 'nome': 'Algoritmos',   'prof': 'Lucidio','alunos': 55}  # Conflito Lucidio + Sala Gigante
]

print(f"Dados configurados: Armadilha de Backtracking pronta.")
print(f"Salas: {len(SALAS)} | Horários: {len(HORARIOS)} | Disciplinas: {len(DISCIPLINAS)}")

Dados configurados: Armadilha de Backtracking pronta.
Salas: 6 | Horários: 4 | Disciplinas: 10


In [2]:
# --- 2. MODELAGEM FORMAL (ESTADOS E AÇÕES) ---

def estado_inicial():
    # O estado é uma tupla: (dicionario_alocacoes, lista_ids_disciplinas_restantes)
    # alocacoes: { 'id_disciplina': ('id_sala', 'horario') }
    ids_restantes = tuple([d['id'] for d in DISCIPLINAS])
    return ({}, ids_restantes)

def eh_objetivo(estado):
    _, restantes = estado
    return len(restantes) == 0

def obter_disciplina(id_disc):
    # Função auxiliar para pegar os dados da disciplina pelo ID
    return next(d for d in DISCIPLINAS if d['id'] == id_disc)

def movimento_valido(alocacoes_atuais, disciplina, sala, horario):
    # RESTRIÇÃO 1: Capacidade [cite: 112]
    if sala['cap'] < disciplina['alunos']:
        return False

    # Verifica conflitos com o que já está alocado
    for id_alocada, (id_sala_alocada, horario_alocado) in alocacoes_atuais.items():
        if horario_alocado == horario:
            # RESTRIÇÃO 2: Sala já ocupada
            if id_sala_alocada == sala['id']:
                return False
            
            # RESTRIÇÃO 3: Professor já ocupado (Conflito) [cite: 112]
            disc_alocada = obter_disciplina(id_alocada)
            if disc_alocada['prof'] == disciplina['prof']:
                return False
                
    return True

def gerar_sucessores(estado):
    alocacoes, restantes = estado
    sucessores = []
    
    if not restantes:
        return []
    
    # Pega a próxima disciplina da lista para tentar alocar
    id_prox = restantes[0]
    novas_restantes = restantes[1:] # Remove a disciplina da lista
    dados_disc = obter_disciplina(id_prox)
    
    # Tenta todas as combinações de sala e horário (Ações Possíveis) [cite: 115]
    for sala in SALAS:
        for horario in HORARIOS:
            if movimento_valido(alocacoes, dados_disc, sala, horario):
                # Cria nova alocação copiando a anterior
                novas_alocacoes = alocacoes.copy()
                novas_alocacoes[id_prox] = (sala['id'], horario)
                
                # Novo estado
                sucessores.append((novas_alocacoes, novas_restantes))
                
    return sucessores

In [3]:
# --- 3. ALGORITMOS DE BUSCA ---

# HEURÍSTICA: Número de disciplinas restantes [cite: 118]
# Quanto menos faltam, mais perto do fim.
def heuristica(estado):
    _, restantes = estado
    return len(restantes)

# --- ESTRATÉGIA 1: A* (Informada) [cite: 13] ---
def busca_a_star(inicio):
    # Fila de prioridade: (f_score, contador_desempate, estado)
    fronteira = []
    contador = 0 
    
    # g = 0 (custo atual), h = disciplinas restantes
    heapq.heappush(fronteira, (heuristica(inicio), contador, inicio))
    
    visitados = 0
    tempo_ini = time.time()
    
    while fronteira:
        _, _, estado_atual = heapq.heappop(fronteira)
        visitados += 1
        
        # Teste de Objetivo [cite: 10]
        if eh_objetivo(estado_atual):
            return estado_atual, visitados, time.time() - tempo_ini
            
        # Custo g é o número de alocações feitas (len da dict de alocacoes)
        alocacoes, _ = estado_atual
        g = len(alocacoes) + 1 
        
        for sucessor in gerar_sucessores(estado_atual):
            h = heuristica(sucessor)
            f = g + h
            contador += 1
            heapq.heappush(fronteira, (f, contador, sucessor))
            
    return None, visitados, time.time() - tempo_ini

# --- ESTRATÉGIA 2: Híbrida BFS + DFS (Sem Informação explícita) [cite: 14] ---
def busca_hibrida(inicio, limite_bfs=2):
    # Usa uma lista simples como deque
    fronteira = [inicio]
    visitados = 0
    tempo_ini = time.time()
    
    while fronteira:
        # HIBRIDIZAÇÃO[cite: 123]:
        # Se alocamos poucas coisas (profundidade < limite), usa BFS (tira do início 0)
        # Se já alocamos bastante, mergulha com DFS (tira do final -1)
        
        profundidade_atual = len(fronteira[0][0]) # tamanho do dict alocacoes do 1º da fila
        
        if profundidade_atual < limite_bfs:
            estado_atual = fronteira.pop(0) # BFS
        else:
            estado_atual = fronteira.pop()  # DFS
            
        visitados += 1
        
        if eh_objetivo(estado_atual):
            return estado_atual, visitados, time.time() - tempo_ini
            
        # Adiciona sucessores ao final
        for sucessor in gerar_sucessores(estado_atual):
            fronteira.append(sucessor)
            
    return None, visitados, time.time() - tempo_ini

In [4]:
# --- 4. EXPERIMENTOS E RESULTADOS ---

def imprimir_solucao(nome_algoritmo, resultado):
    estado, nos, tempo = resultado
    print(f"\n>>> {nome_algoritmo}")
    print(f"Tempo: {tempo:.4f}s | Nós gerados: {nos}")
    
    if estado:
        alocacoes, _ = estado
        print(f"{'DISCIPLINA':<12} | {'SALA':<5} | {'HORÁRIO'}")
        print("-" * 35)
        for d_id, (s_id, hor) in alocacoes.items():
            nome_disc = obter_disciplina(d_id)['nome']
            print(f"{nome_disc:<12} | {s_id:<5} | {hor}")
    else:
        print("Nenhuma solução encontrada.")

# Rodando...
inicio = estado_inicial()

# Executa A*
res_a = busca_a_star(inicio)
imprimir_solucao("A* (Heurística: nº disciplinas restantes)", res_a)

# Executa Híbrido
# limite_bfs=2 significa: tenta todas as combinações para as 2 primeiras disciplinas (BFS),
# depois tenta fechar a grade o mais rápido possível (DFS).
res_h = busca_hibrida(inicio, limite_bfs=2)
imprimir_solucao("Híbrido (BFS nas 2 primeiras fases -> DFS)", res_h)

KeyboardInterrupt: 